In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_prot_fin = pd.read_csv('/content/drive/My Drive/Colab Notebooks/twenty_five.csv')

In [6]:
df_prot_fin.head()

,Unnamed: 0,structureId,classification,sequence,chainId,seq_length
0,67,117E,HYDROLASE,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,A,286
1,68,117E,HYDROLASE,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,B,286
2,74,11AS,LIGASE,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,A,330
3,75,11AS,LIGASE,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,B,330
4,76,11BA,HYDROLASE,KESAAAKFERQHMDSGNSPSSSSNYCNLMMCCRKMTQGKCKPVNTF...,A,124


### Keras Sequential

### Using tfidf vectorization for the implementation of the model

### Train-Test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test,y_train,y_test = train_test_split(df_prot_fin['sequence'], df_prot_fin['classification'], test_size = 0.3, 
                                                  random_state = 101)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf_vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range= (4,4))

In [11]:
tfidf_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [0]:
vector = tfidf_vectorizer.transform(X_train)

In [0]:
vector1 = tfidf_vectorizer.transform(X_test)

In [14]:
print(vector.shape)

(145946, 170441)


In [0]:
input_dim = (vector.shape[1])
#print(vector1.shape)

In [16]:
y_train.head()

207370                        HYDROLASE
146864    HYDROLASE/HYDROLASE INHIBITOR
78476                       TRANSFERASE
80353                         HYDROLASE
16159                 SIGNALING PROTEIN
Name: classification, dtype: object

### Since Keras takes only integers for the training, I am going to convert the y features into a one hot encoded label

In [17]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


## Encoding y_train and y_test

In [0]:
encoder = LabelEncoder()

encoder.fit(y_train)
encoded_y = encoder.transform(y_train)

dummy_ytrain = np_utils.to_categorical(encoded_y)

In [0]:
encoder1 = LabelEncoder()

encoder1.fit(y_test)
encoded_y1 = encoder1.transform(y_test)

dummy_ytest = np_utils.to_categorical(encoded_y1)

In [20]:
dummy_ytrain[0:4]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

### Building a sequential model from kera with a dense layer

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [22]:
model = Sequential()
model.add(Dense(units = 128, activation = "relu", input_dim=input_dim)) # the numbers of features as the input_dim

model.add(Dense(units= 15, activation = "softmax"))
          

Instructions for updating:
Colocations handled automatically by placer.


### Since there are more than 2 classes, I hve chosen softmax as the final ouput layer. 

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               21816576  
_________________________________________________________________
dense_2 (Dense)              (None, 15)                1935      
Total params: 21,818,511
Trainable params: 21,818,511
Non-trainable params: 0
_________________________________________________________________


### I have chosen a validation set from the training data and compare the accuracy with the accuracy of the training data. The model seems to have a high accuracy for the training sets of data as compared to validation sets, which generally implies that the model is overfitting and is not performing well with the unseen data

In [24]:
history = model.fit(vector, dummy_ytrain,
epochs=5,
verbose=1,
validation_split=0.33,
batch_size=128)

Instructions for updating:
Use tf.cast instead.
Train on 97783 samples, validate on 48163 samples
Epoch 1/5
97783/97783 [==============================] - 164s 2ms/step - loss: 0.8181 - acc: 0.7804 - val_loss: 0.3814 - val_acc: 0.8888
Epoch 2/5
97783/97783 [==============================] - 161s 2ms/step - loss: 0.2165 - acc: 0.9343 - val_loss: 0.3395 - val_acc: 0.8935
Epoch 3/5
97783/97783 [==============================] - 160s 2ms/step - loss: 0.1671 - acc: 0.9433 - val_loss: 0.3377 - val_acc: 0.8964
Epoch 4/5
97783/97783 [==============================] - 160s 2ms/step - loss: 0.1514 - acc: 0.9470 - val_loss: 0.3342 - val_acc: 0.8981
Epoch 5/5
97783/97783 [==============================] - 160s 2ms/step - loss: 0.1411 - acc: 0.9496 - val_loss: 0.3331 - val_acc: 0.8988


In [0]:
from sklearn.metrics import accuracy_score

**Evaluating the accuracy  for the test vectorized data and we can see that the accuracy has increased as compared to validation data but reamins low as compared to training data. This implies overfitting, even though the accuracy is on the higher side**

In [31]:
scores = model.evaluate(vector1, dummy_ytest, verbose=1)

62549/62549 [==============================] - 62s 996us/step


In [32]:
print(scores)

[0.334197763473119, 0.900014388720197]


### I am going to set up the test dataset as a validation data and run the model again. Doing so, the accuacy has increased but the still remains a lower than the training dataset, which again implies overfitting

In [35]:
history = model.fit(vector, dummy_ytrain,

epochs=5,
verbose=1,
validation_data=(vector1, dummy_ytest),
batch_size=128)

Train on 145946 samples, validate on 62549 samples
Epoch 1/5
145946/145946 [==============================] - 228s 2ms/step - loss: 0.1857 - acc: 0.9380 - val_loss: 0.2621 - val_acc: 0.9179
Epoch 2/5
145946/145946 [==============================] - 227s 2ms/step - loss: 0.1343 - acc: 0.9512 - val_loss: 0.2660 - val_acc: 0.9186
Epoch 3/5
145946/145946 [==============================] - 227s 2ms/step - loss: 0.1283 - acc: 0.9527 - val_loss: 0.2648 - val_acc: 0.9184
Epoch 4/5
145946/145946 [==============================] - 227s 2ms/step - loss: 0.1217 - acc: 0.9546 - val_loss: 0.2659 - val_acc: 0.9198
Epoch 5/5
145946/145946 [==============================] - 229s 2ms/step - loss: 0.1178 - acc: 0.9554 - val_loss: 0.2589 - val_acc: 0.9207


### This model can be improved 

* Thisngs to consuder in the future may be when tfidf vectorization is carried out, introduce some restriction on the number of sequence

* The number of layers can be tested with more than one, 

* The number of neurons in the hidden layer needs to be optimized. There doesnot seem to be a solid answer for how many neurons should be there. I have tried 100, 200, 500, all seems to yield more or near th same number

May be used some regulariation should be done, such as using drop out method.